In [116]:
import time

from enum import Enum
class OrderType(Enum):
    LIMIT = 1
    MARKET = 2
    IOC = 3

class OrderSide(Enum):
    BUY = 1
    SELL = 2


class NonPositiveQuantity(Exception):
    pass

class NonPositivePrice(Exception):
    pass

class InvalidSide(Exception):
    pass

class UndefinedOrderType(Exception):
    pass

class UndefinedOrderSide(Exception):
    pass

class NewQuantityNotSmaller(Exception):
    pass

class UndefinedTraderAction(Exception):
    pass

class UndefinedResponse(Exception):
    pass


from abc import ABC


class Order(ABC):
    def __init__(self, id, symbol, quantity, side, time):
        self.id = id
        self.symbol = symbol
        if quantity > 0:
            self.quantity = quantity
        else:
            raise NonPositiveQuantity("Quantity Must Be Positive!")
        if side in [OrderSide.BUY, OrderSide.SELL]:
            self.side = side
        else:
            raise InvalidSide("Side Must Be Either \"Buy\" or \"OrderSide.SELL\"!")
        self.time = time


class LimitOrder(Order):
    def __init__(self, id, symbol, quantity, price, side, time):
        super().__init__(id, symbol, quantity, side, time)
        if price > 0:
            self.price = price
        else:
            raise NonPositivePrice("Price Must Be Positive!")
        self.type = OrderType.LIMIT


class MarketOrder(Order):
    def __init__(self, id, symbol, quantity, side, time):
        super().__init__(id, symbol, quantity, side, time)
        self.type = OrderType.MARKET


class IOCOrder(Order):
    def __init__(self, id, symbol, quantity, price, side, time):
        super().__init__(id, symbol, quantity, side, time)
        if price > 0:
            self.price = price
        else:
            raise NonPositivePrice("Price Must Be Positive!")
        self.type = OrderType.IOC
    

class FilledOrder(Order):
    def __init__(self, id, symbol, quantity, price, side, time, limit = False):
        super().__init__(id, symbol, quantity, side, time)
        self.price = price
        self.limit = limit
        



class MatchingEngine():
    def __init__(self):
        self.bid_book = []
        self.ask_book = []
        # These are the order books you are given and expected to use for matching the orders below

    # Note: As you implement the following functions keep in mind that these enums are available:
#     class OrderType(Enum):
#         LIMIT = 1
#         MARKET = 2
#         IOC = 3

#     class OrderSide(Enum):
#         BUY = 1
#         SELL = 2

    def handle_order(self, order):
        # Implement this function
        # In this function you need to call different functions from the matching engine
        # depending on the type of order you are given
        
        if order.type == OrderType.LIMIT:
            handle_limit_order(order)
        elif order.type == OrderType.MARKET:
            handle_market_order(order)
        elif order.type == OrderType.IOC:
            handle_ioc_order(order)
        else:
            raise UndefinedOrderType("Undefined Order Type!")
        
        # You need to raise the following error if the type of order is ambiguous
        #raise UndefinedOrderType("Undefined Order Type!")
        
    
    def handle_limit_order(self, order): 
        # Implement this function
        # Keep in mind what happens to the orders in the limit order books when orders get filled
        # or if there are no crosses from this order
        # in other words, handle_limit_order accepts an arbitrary limit order that can either be 
        # filled if the limit order price crosses the book, or placed in the book. If the latter, 
        # pass the order to insert_limit_order below. 
        if order.side not in [OrderSide.BUY, OrderSide.SELL]:
            raise UndefinedOrderSide("Undefined Order Side!")
        
        filled_orders = []
        
        """For this we should probably insert into the ask or bid book first, sort by price and then check to see if 
        the best ask and bid are good for each other then from there do the transaction and then worry about the quantity
        if limit order quantity < order in book quantity move to the next order in the book after amending the quantity
        to make it equal to the shares waiting to be filled and then check to see if the price of the next order is good
        if yes """
        from copy import deepcopy
        
        cantrade = True
        
        if order.side == OrderSide.BUY:
            if len(self.ask_book) == 0:
                cantrade = False
                self.insert_limit_order(order)
        elif order.side == OrderSide.SELL:
            if len(self.bid_book) == 0:
                cantrade = False
                self.insert_limit_order(order)
                
        if not cantrade:
            return filled_orders 
        #the above takes care of the first part of the test case in a rather clever way if i may say so myself
        
        #from here we can assume that we have to make a transaction occur
        
        def getprice(order):
            return order.price, order.time
        
        self.bid_book.sort(reverse = True, key = getprice)
        self.ask_book.sort(key = getprice)
        
        if order.side == OrderSide.BUY:
            book = self.ask_book
            #shareswanted = deepcopy(order.quantity) #amount of shares you want to buy
            #the cases are: perfect matching in quantity and everything is fill just fill both orders
            #imperfect matching case 1: wtb/s more than available. fill all valid opposite orders put order
            #with quantity of the amount you still want in respective book
            #imperfect matching case 2: more available than you want. Fill  the partial order and then 
            for i in book:
                if order.price < i.price: #willing to buy for less than they're asking
                    #sharesavail = deepcopy(i.quantity) #amount of shares youre buying this time
                    #sharestofill = sharesavail - shareswanted #might be negative 
                    #order.quantity = sharesavail
                    if i.quantity > order.quantity:
                        amount = deepcopy(order.quantity)
                        x = FilledOrder(order.id, order.symbol, amount, order.price, order.side, order.time, limit = True)
                        y = FilledOrder(i.id, i.symbol, amount, i.price, i.side, i.time, limit = True)
                        filled_orders.append(x) #this fills the full buy order 
                        filled_orders.append(y) #this fills the partial sale order 
                        i.quantity -= amount 
                        break
                        
                        
                    elif i.quantity < order.quantity:
                        amount = deepcopy(i.quantity)
                        x = FilledOrder(i.id, i.symbol, amount, i.price, i.side, i.time, limit = True)
                        y = FilledOrder(order.id, order.symbol, amount, order.price, order.side, order.time, limit = True)
                        filled_orders.append(x) #fills the full sale order
                        filled_orders.append(y) #this fills the partial buy order 
                        order.quantity -= deepcopy(i.quantity)
                        idx = self.ask_book.index(i)
                        self.ask_book.pop(idx)
                        
                    else:
                        amount = deepcopy(i.quantity)
                        x = FilledOrder(i.id, i.symbol, amount, i.price, i.side, i.time, limit = True)
                        y = FilledOrder(order.id, order.symbol, amount, order.price, order.side, order.time, limit = True)
                        filled_orders.append(x)
                        filled_orders.append(y)
                        idx = self.ask_book.index(i)
                        self.ask_book.pop(idx)
                        
                    
                
        else:
            book = self.bid_book
            for i in self.bid_book:
                if order.price < i.price: #willing to sell for less than they offer
                    #sharesavail = deepcopy(i.quantity) #amount of shares youre buying this time
                    #sharestofill = sharesavail - shareswanted #might be negative 
                    #order.quantity = sharesavail
                    if i.quantity > order.quantity:
                        amount = deepcopy(order.quantity)
                        x = FilledOrder(order.id, order.symbol, amount, order.price, order.side, order.time, limit = True)
                        y = FilledOrder(i.id, i.symbol, amount, i.price, i.side, i.time, limit = True)
                        filled_orders.append(y) #this fills the partial sale order 
                        filled_orders.append(x) #this fills the full buy order 
                        #filled_orders.append(y) #this fills the partial sale order 
                        i.quantity -= amount 
                        break

                    elif i.quantity < order.quantity:
                        amount = deepcopy(i.quantity)
                        x = FilledOrder(i.id, i.symbol, amount, i.price, i.side, i.time, limit = True)
                        y = FilledOrder(order.id, order.symbol, amount, order.price, order.side, order.time, limit = True)
                        filled_orders.append(x) #fills the full sale order
                        filled_orders.append(y) #this fills the partial buy order 
                        
                        order.quantity -= deepcopy(i.quantity)
                        idx = self.bid_book.index(i)
                        self.bid_book.pop(idx)

                    else:
                        amount = deepcopy(i.quantity)
                        x = FilledOrder(i.id, i.symbol, amount, i.price, i.side, i.time, limit = True)
                        y = FilledOrder(order.id, order.symbol, amount, order.price, order.side, order.time, limit = True)
                        filled_orders.append(x)
                        filled_orders.append(y)
                        idx = self.bid_book.index(i)
                        self.bid_book.pop(idx)

                """for i in book:
                    if order.price < i.price:
                        sharesavail = deepcopy(i.quantity) #amount of shares youre buying this time
                        shareswanted = deepcopy(order.quantity) #amount of shares you want to buy
                        sharestofill = sharesavail - shareswanted #might be negative 
                        order.quantity = sharesavail
                        #if i.quantity > order.quantity:

                        filled_orders.append(i)
                        filled_orders.append(order)
                        idx = book.index(i)
                        book.pop(idx) #This should pop the index for the order in the class book due to pointers
                        if sharestofill < 0:
                            order.quantity = -sharestofill
                        elif sharestofill == 0:
                            break"""

        # The orders that are filled from the limit order need to be inserted into the above list
        # The filled orders are expected to be the return variable (list)
        if len(filled_orders)  == 0:
            self.insert_limit_order(order)
        
        def getprice(order):
            return order.price, order.time
        
        self.bid_book.sort(key = getprice)
        self.ask_book.sort(key = getprice)
        
        return filled_orders
        
        # You need to raise the following error if the side the order is for is ambiguous
        #raise UndefinedOrderSide("Undefined Order Side!")


    def handle_market_order(self, order):
        if order.side not in [OrderSide.BUY, OrderSide.SELL]:
            raise UndefinedOrderSide("Undefined Order Side!")
            
        from copy import deepcopy
            
        # Implement this function
        filled_orders = []
        # The orders that are filled from the market order need to be inserted into the above list
        
        def fifo(order):
            return order.price,order.time
        
        if order.side == OrderSide.BUY:
            self.ask_book.sort(key = fifo)
            for i in book:
                price = deepcopy(i.price)
                if i.quantity > order.quantity:
                    amount = deepcopy(order.quantity)
                    x = FilledOrder(order.id, order.symbol, amount, price, order.side, order.time, limit = True)
                    y = FilledOrder(i.id, i.symbol, amount, price, i.side, i.time, limit = True)
                    filled_orders.append(y) #this fills the partial sale order 
                    filled_orders.append(x) #this fills the full buy order 
                    #filled_orders.append(y) #this fills the partial sale order 
                    i.quantity -= amount 
                    break
                elif i.quantity < order.quantity:
                    amount = deepcopy(i.quantity)
                    x = FilledOrder(i.id, i.symbol, amount, price, i.side, i.time, limit = True)
                    y = FilledOrder(order.id, order.symbol, amount, price, order.side, order.time, limit = True)
                    filled_orders.append(x) #fills the full sale order
                    filled_orders.append(y) #this fills the partial buy order 

                    order.quantity -= deepcopy(i.quantity)
                    idx = self.ask_book.index(i)
                    self.ask_book.pop(idx)

                else:
                    amount = deepcopy(i.quantity)
                    x = FilledOrder(i.id, i.symbol, amount, price, i.side, i.time, limit = True)
                    y = FilledOrder(order.id, order.symbol, amount, price, order.side, order.time, limit = True)
                    filled_orders.append(x)
                    filled_orders.append(y)
                    idx = self.bid_book.index(i)
                    self.bid_book.pop(idx)
                    
        else:
            self.bid_book.sort(key = fifo)
            book = self.bid_book
            for i in book:
                price = deepcopy(i.price)
                if i.quantity > order.quantity:
                    amount = deepcopy(order.quantity)
                    x = FilledOrder(order.id, order.symbol, amount, price, order.side, order.time, limit = True)
                    y = FilledOrder(i.id, i.symbol, amount, price, i.side, i.time, limit = True)
                    filled_orders.append(y) #this fills the partial sale order 
                    filled_orders.append(x) #this fills the full buy order 
                    #filled_orders.append(y) #this fills the partial sale order 
                    i.quantity -= amount 
                    break

                elif i.quantity < order.quantity:
                    amount = deepcopy(i.quantity)
                    x = FilledOrder(i.id, i.symbol, amount, price, i.side, i.time, limit = True)
                    y = FilledOrder(order.id, order.symbol, amount, price, order.side, order.time, limit = True)
                    filled_orders.append(x) #fills the full sale order
                    filled_orders.append(y) #this fills the partial buy order 

                    order.quantity -= deepcopy(i.quantity)
                    idx = self.bid_book.index(i)
                    self.bid_book.pop(idx)

                else:
                    amount = deepcopy(i.quantity)
                    x = FilledOrder(i.id, i.symbol, amount, price, i.side, i.time, limit = True)
                    y = FilledOrder(order.id, order.symbol, amount, price, order.side, order.time, limit = True)
                    filled_orders.append(x)
                    filled_orders.append(y)
                    idx = self.bid_book.index(i)
                    self.bid_book.pop(idx)
        
        # The filled orders are expected to be the return variable (list)
        return filled_orders
        
        # You need to raise the following error if the side the order is for is ambiguous
        #raise UndefinedOrderSide("Undefined Order Side!")
        

    def handle_ioc_order(self, order):
        if order.side not in [OrderSide.BUY, OrderSide.SELL]:
            raise UndefinedOrderSide("Undefined Order Side!")
            
        # Implement this function
        filled_orders = []
        # The orders that are filled from the ioc order need to be inserted into the above list
        from copy import deepcopy
        
        if order.side == OrderSide.BUY:
            #self.ask_book.sort(key = fifo)
            for i in self.ask_book:
                if order.price < i.price:
                    if order.quantity == i.quantity:
                        amount = deepcopy(i.quantity)
                        x = FilledOrder(i.id, i.symbol, amount, i.price, i.side, i.time, limit = True)
                        y = FilledOrder(order.id, order.symbol, amount, order.price, order.side, order.time, limit = True)
                        filled_orders.append(x)
                        filled_orders.append(y)
                        idx = self.asl_book.index(i)
                        self.ask_book.pop(idx)
        else:
            for i in self.bid_book:
                if order.price < i.price:
                    if order.quantity == i.quantity:
                        amount = deepcopy(i.quantity)
                        x = FilledOrder(i.id, i.symbol, amount, i.price, i.side, i.time, limit = True)
                        y = FilledOrder(order.id, order.symbol, amount, order.price, order.side, order.time, limit = True)
                        filled_orders.append(x)
                        filled_orders.append(y)
                        idx = self.bid_book.index(i)
                        self.bid_book.pop(idx)
                
        
        # The filled orders are expected to be the return variable (list)
        return filled_orders
        
        # You need to raise the following error if the side the order is for is ambiguous
        #raise UndefinedOrderSide("Undefined Order Side!")


    def insert_limit_order(self, order):
        assert order.type == OrderType.LIMIT
        if order.side not in [OrderSide.BUY, OrderSide.SELL]:
            raise UndefinedOrderSide("Undefined Order Side!")
        # Implement this function
        # this function's sole puporse is to place limit orders in the book that are guaranteed
        # to not immediately fill
        def getprice(order):
            return order.price
        
        if order.side == OrderSide.BUY:
            self.bid_book.append(order)
            self.bid_book.sort(reverse = True, key = getprice)
        else:
            self.ask_book.append(order)
            self.ask_book.sort(key = getprice)
        
        
        # You need to raise the following error if the side the order is for is ambiguous
        #raise UndefinedOrderSide("Undefined Order Side!")

    def amend_quantity(self, id, quantity):
        # Implement this function
        # Hint: Remember that there are two order books, one on the bid side and one on the ask side
        for i in self.ask_book:
            if i.id == id:
                order = i
                break
        for i in self.bid_book:
            if i.id == id:
                order = i
                break
        if quantity < order.quantity:
            order.quantity = quantity
        else:
            # You need to raise the following error if the user attempts to modify an order
            # with a quantity that's greater than given in the existing order
            raise NewQuantityNotSmaller("Amendment Must Reduce Quantity!")
        
    
    def cancel_order(self, id):
        # Implement this function 
        # Think about the changes you need to make in the order book based on the parameters given
        self.bid_book = [i for i in self.bid_book if i.id != id]
        self.ask_book = [i for i in self.ask_book if i.id != id]
    




In [67]:
matching_engine.bid_book.sort(reverse = False, key = getprice)

In [70]:
#matching_engine.bid_book
len(matching_engine.bid_book)

1

In [63]:
def getprice(order):
    return order.price, order.time

In [10]:
x = matching_engine.bid_book
for i in x:
    print(i)

In [11]:
x.sort(reverse = True, key = getprice)
for i in x:
    print(i.price)

15
10
10


In [109]:
#def test_handle_ioc_order(self):
matching_engine = MatchingEngine()
order_1 = LimitOrder(1, "S", 1, 10, OrderSide.BUY, time.time())
order_2 = LimitOrder(2, "S", 5, 10, OrderSide.BUY, time.time())
matching_engine.handle_limit_order(order_1)
matching_engine.handle_limit_order(order_2)

order = IOCOrder(6, "S", 5, 12, OrderSide.SELL, time.time())
filled_orders = matching_engine.handle_ioc_order(order)
assert(matching_engine.bid_book[0].quantity == 1)
assert(len(filled_orders) == 0)

In [107]:
filled_orders[1]

In [117]:
matching_engine = MatchingEngine()
order_1 = LimitOrder(1, "S", 1, 10, OrderSide.BUY, time.time())
order_2 = LimitOrder(2, "S", 5, 10, OrderSide.BUY, time.time())
matching_engine.handle_limit_order(order_1)
matching_engine.handle_limit_order(order_2)

order = IOCOrder(6, "S", 5, 12, OrderSide.SELL, time.time())
filled_orders = matching_engine.handle_ioc_order(order)
assert(matching_engine.bid_book[0].quantity == 1)
assert(len(filled_orders) == 0)

In [118]:
matching_engine.bid_book[0].quantity

1

In [119]:
len(filled_orders)

0